In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
import matplotlib.ticker as ticker
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn import preprocessing

import json
from pandas.io.json import json_normalize

%matplotlib inline

import folium # map rendering library

## scraping data from website out of scope to this week lesson
Copied data and cleaned in csv and read here


In [2]:
df = pd.read_csv('Canada_postcode_M.csv')
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df['PostalCode'].head()

0    M3A
1    M4A
2    M5A
3    M6A
4    M7A
Name: PostalCode, dtype: object

In [4]:
df.shape

(103, 3)

To  utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

In [5]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# postal_code='M5A'
# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

In [6]:
ll_df = pd.read_csv('Geospatial_Coordinates.csv')
M_df=pd.merge(df,ll_df,on=['PostalCode'])

In [9]:
# Filter Toronto:
toronto_df=M_df.loc[ M_df['Borough'].str.contains("Toronto")]
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [12]:
neighborhoods=toronto_df.drop(['PostalCode'],1)

# Toronto coordinate:
latitude = 43.6532 
longitude=-79.3832

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
M_df.loc[ (M_df['Latitude'] > 43.62) &  (M_df['Latitude'] < 43.68) & (M_df['Longitude'] > -79.4) & (M_df['Longitude'] < -79.3)]
# # Toronto coordinate:
# latitude = 43.6532 
# longitude=-79.3832

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [20]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


In [21]:
neighborhood_name = 'Rosedale'
df= M_df.loc[ M_df['Neighborhood']==neighborhood_name].reset_index()

neighborhood_latitude = df.loc[0, 'Latitude']
neighborhood_longitude = df.loc[0, 'Longitude'] 
print(neighborhood_longitude)

-79.37752940000001


In [22]:
# @hidden_cell

CLIENT_ID = 'XERQ2FZVJQJWDBOUFZOMM0USX0A4T4DTZRPVC032SDE2ZLOA' # your Foursquare ID
CLIENT_SECRET = 'BNG1Y3XLVJF5ZMFROXQRZ2Z0SV5JO1LRMZXGPKT0ZFVYIPVE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
    

In [23]:
import requests 

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [24]:
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    500, 
    100)

results = requests.get(url).json()

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## Explore Neighborhoods in Downtown Toronto

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=30):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# Don't run repeatly!

# Make Call to oursquare.com
toronto_venues = getNearbyVenues(names=toronto_df ['Neighborhood'],
                                   latitudes=toronto_df ['Latitude'],
                                   longitudes=toronto_df ['Longitude']
                                  )


NameError: name 'manhattan_venues' is not defined

In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 195 uniques categories.


In [38]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


## Analyze Each Neighborhood

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Get the mean of frequency of occurence of venue category in each neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(toronto_onehot.shape)
print(toronto_grouped.shape)
toronto_grouped.head()

(865, 195)
(39, 195)


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.117647,0.176471,0.117647,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0


#### What are the top 10 most common venue categories in each hood ?

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bistro,Café,Basketball Stadium,Jazz Club,Breakfast Spot,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Pet Store,Bakery,Performing Arts Venue,Convenience Store,Climbing Gym,Restaurant,Burrito Place
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Fast Food Restaurant,Skate Park,Brewery,Smoke Shop,Light Rail Station,Spa,Restaurant,Recording Studio,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Plane,Harbor / Marina,Bar
4,Central Bay Street,Coffee Shop,Café,Yoga Studio,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Modern European Restaurant


## Cluster Neighborhoods

In [46]:
from sklearn.cluster import KMeans

In [71]:
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop(['Cluster Labels'],1)

In [72]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [73]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Yoga Studio,Restaurant,Ice Cream Shop,Café,Pub
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Wings Joint,Discount Store,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Gym,Fried Chicken Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Theater,Coffee Shop,Sporting Goods Shop,Plaza,Burrito Place,Electronics Store,Bar,Hotel,Shopping Mall
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Café,Coffee Shop,Restaurant,BBQ Joint,Cocktail Bar,Middle Eastern Restaurant,Cosmetics Shop,Creperie,Poke Place
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Pub,Health Food Store,Wings Joint,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
# cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Theater,Breakfast Spot,Yoga Studio,Restaurant,Ice Cream Shop,Café,Pub
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Wings Joint,Discount Store,Park,Mexican Restaurant,Italian Restaurant,Hobby Shop,Gym,Fried Chicken Joint
9,"Garden District, Ryerson",Café,Theater,Coffee Shop,Sporting Goods Shop,Plaza,Burrito Place,Electronics Store,Bar,Hotel,Shopping Mall
15,St. James Town,Gastropub,Café,Coffee Shop,Restaurant,BBQ Joint,Cocktail Bar,Middle Eastern Restaurant,Cosmetics Shop,Creperie,Poke Place
20,Berczy Park,Beer Bar,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bistro,Café,Basketball Stadium,Jazz Club,Breakfast Spot,Restaurant
24,Central Bay Street,Coffee Shop,Café,Yoga Studio,Seafood Restaurant,Sandwich Place,Bubble Tea Shop,Ramen Restaurant,Poke Place,Chinese Restaurant,Modern European Restaurant
25,Christie,Grocery Store,Café,Park,Coffee Shop,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports
30,"Richmond, Adelaide, King",Coffee Shop,Café,Steakhouse,Sushi Restaurant,Deli / Bodega,Seafood Restaurant,Hotel,Plaza,Smoke Shop,Speakeasy
31,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Portuguese Restaurant,Supermarket,Brewery,Music Venue,Smoke Shop,Café,Bar,Bank
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Japanese Restaurant,Restaurant,Deli / Bodega,Train Station,Gym / Fitness Center,Bakery,Sandwich Place,Hotel


In [63]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Rosedale,Park,Playground,Trail,Wings Joint,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


In [64]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] ==2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,The Beaches,Trail,Pub,Health Food Store,Wings Joint,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
36,"Harbourfront East, Union Station, Toronto Islands",Park,Hotel,Plaza,Bubble Tea Shop,Skating Rink,IT Services,Italian Restaurant,Japanese Restaurant,Salad Place,Roof Deck
37,"Little Portugal, Trinity",Asian Restaurant,Bar,Yoga Studio,Malay Restaurant,Boutique,Brewery,Record Shop,Pizza Place,Coffee Shop,New American Restaurant
41,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Bubble Tea Shop,Bookstore,Restaurant,Spa,Dessert Shop
47,"India Bazaar, The Beaches West",Park,Fast Food Restaurant,Movie Theater,Pet Store,Pizza Place,Pub,Liquor Store,Light Rail Station,Burrito Place,Restaurant
61,Lawrence Park,Bus Line,Park,Swim School,Wings Joint,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
67,Davisville North,Park,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Hotel,Dance Studio,Convenience Store,Cosmetics Shop
68,"Forest Hill North & West, Forest Hill Road Park",Trail,Sushi Restaurant,Mexican Restaurant,Jewelry Store,Wings Joint,Deli / Bodega,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center
75,"Parkdale, Roncesvalles",Gift Shop,Breakfast Spot,Bar,Italian Restaurant,Eastern European Restaurant,Dog Run,Dessert Shop,Cuban Restaurant,Movie Theater,Restaurant
86,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Pizza Place,Restaurant,Bank,Light Rail Station,Bagel Shop,Sports Bar,Liquor Store,Supermarket


In [65]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,"Moore Park, Summerhill East",Restaurant,Park,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store


In [66]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,Home Service,Garden,Ice Cream Shop,Dance Studio,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
